In [10]:
from myutils.DB import DB
import myutils.my_utils as my
import time,FinanceDataReader as fdr

class SU :

    def __init__(self) :
        self.DB = DB('stocks')
        self.skey, self.appkey = self.DB.exe("SELECT p_data_01,p_data_02 FROM my_keep_data WHERE no=1",many=1,assoc=False)

    def stocks_update(self) :

        today = my.timestamp_to_date(opt=7)
        week_day = my.dayofdate(today)

        if week_day in ['일','월'] :
            message = f"[{today}] {week_day}요일 : Good morning !"
            my.post_slack(self.skey,message)
            return

        self.update_fdr('soxl')
        self.update_fdr('jepq')

        self.DB.close()

        message = f"[{today}] 주가 정보 업데이트를 완료하였습니다"
        my.post_slack(self.skey,message)

    def update_fdr(self,cdx) :
        cdx = cdx.upper()
        self.DB.tbl, self.DB.wre = ('h_stockHistory_board',f"add1='{cdx}'")
        b_date = self.DB.get("max(add0)",many=1,assoc=False)

        df = fdr.DataReader(cdx,start=b_date)
        df['Str_Date'] = df.index.strftime('%Y-%m-%d')

        df['Up']     = 0
        df['Dn']     = 0

        df['Open']  = round(df['Open'],2)
        df['High']  = round(df['High'],2)
        df['Low']   = round(df['Low'],2)
        df['Close'] = round(df['Close'],2)
        df['Change']= round(df['Close'].diff(periods=1)/df['Close'].shift(1),4)

        df = df[['Str_Date','Open','High','Low','Close','Volume','Change','Up','Dn']]
        dflist = df.values.tolist()

        self.DB.wre = f"add0='{b_date}' and add1='{cdx}'"
        one = self.DB.get('add9,add10',many=1,assoc=False)

        dflist[0][7] = int(one[0])
        dflist[0][8] = int(one[1])

        for i in range(1,len(dflist)) :
            dflist[i][7]  = dflist[i-1][7]+1 if dflist[i][4] >= dflist[i-1][4] else 0
            dflist[i][8]  = dflist[i-1][8]+1 if dflist[i][4] <  dflist[i-1][4] else 0

        ohlc = dflist[1:]

        db_keys = "add0,add4,add5,add6,add3,add7,add8,add9,add10,add1,add2,uid,uname,wdate,mdate"
        time_now = my.now_timestamp()

        print(ohlc)

        # for row in ohlc :
        #     row2 = list(row)
        #     row2 += [cdx,cdx,'comphys','정용훈',time_now,time_now]
        #     values = str(row2)[1:-1]
        #     sql = f"INSERT INTO {self.DB.tbl} ({db_keys}) VALUES({values})"
        #     self.DB.exe(sql)


my_stocks = SU()
my_stocks.stocks_update()

[['2023-01-31', 13.67, 14.55, 13.64, 14.52, 97363342, 0.0522, 1, 0]]
[['2023-01-31', 43.05, 43.35, 43.0, 43.35, 752973, 0.0105, 1, 0]]


In [6]:
from myutils.DB import DB as LDB
import myutils.my_utils as my
import time,FinanceDataReader as fdr

DB = LDB('stocks')
cdx = 'SOXL'
DB.tbl, DB.wre = ('h_stockHistory_board',f"add1='{cdx}'")
b_date = DB.get("max(add0)",many=1,assoc=False)
e_date = my.timestamp_to_date(opt=7)

df = fdr.DataReader(cdx,start=b_date)

df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-30,14.35,14.6200,13.73,13.80,13.80,80855500
2023-01-31,13.67,14.5486,13.64,14.52,14.52,97363342
